# Microsoft Azure Developer Camp
## AI & Machine Learning
---

### Session 2. Cifar10 Image Classification with Keras with GPU 😎

* [Keras Document](http://keras.io/)  
* [Cifar10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)


### 0.GPU 활용

In [ ]:
# Tensorflow Backend에서 사용 가능한 디바이스 확인
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# 특정 디바이스 사용
import keras.backend.tensorflow_backend as K

K.tf.device('/device:GPU:0')

K._get_available_gpus() 

## Part1. Data Preprocessing
### 1. Cifar10 데이터 가져오기

In [ ]:
# 총 60,000장의 이미지로(163MB), 내려받는데 몇 분 정도 소요됩니다.
from keras.datasets import cifar10

# 불러온 데이터를 훈련셋과 검증셋으로 나눕니다.(훈련셋 50,000장, 검증셋 10,000장)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('X_train shape:', X_train.shape)

class_name = ('airplane', 'automobile', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### 2. 데이터 이미지 확인

In [ ]:
import matplotlib.pyplot as plt
import PIL 
import numpy as np

# 데이터 중 일부를 랜덤으로 선택해 이미지와 label 확인
ROW = 3
COLUMN = 3

image_indexes = np.random.choice(len(np.array(X_train)), ROW * COLUMN)

for i in range(ROW * COLUMN):
    label_index = ((y_train[image_indexes][i])[0])
    plt.subplot(ROW, COLUMN, 1+i)
    plt.imshow(PIL.Image.fromarray(X_train[image_indexes][i]))
    plt.title("%s" % class_name[label_index])

plt.tight_layout()
plt.show()
                          

### 3. 데이터 전처리

In [ ]:
# 이미지 전처리 : Normalize 0~255 -> 0~1 

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255
X_test = X_test/255

print('X_train shape:', X_train.shape)

In [ ]:
import keras.utils as utils

# Label 전처리 : One hot encoding

y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

num_classes = y_test.shape[1]

print("\ny_train:\n")
print(y_train.shape)

print("\ny_val:\n")
print(y_test.shape)

print('num_classes:', num_classes)

---

## Part2. Model Train
### 4.CNN 모델 생성

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers

    
# 모델 생성/레이어 쌓기
# model = Sequential()
# model.add(Conv2D(8, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(32, 32, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# 모델 compile
sgd = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

### 5. 모델 훈련

In [ ]:
batch_size = 32
epochs = 100

# 모델 훈련
history = model.fit(X_train, y_train,
                 epochs=epochs,
                 verbose=1,
                 validation_data=(X_test, y_test),
                 shuffle=True)


---

## Part3. Model Evaluate and Save
### 6. 모델 평가 

In [ ]:
# 훈련된 모델 평가
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

### 7. 모델 저장

In [ ]:
# notebook 디렉토리에 모델 저장
model.save('keras_cifar10_trained_model.h5')

---

## Part4. Vsualize Training
### 8. 훈련 과정 살펴보기

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

---

## Part5. Test Model
### 9. 랜덤 이미지로 모델 테스트

In [ ]:
# 검증 셋 중 3장의 이미지를 랜덤으로 선택해 훈련된 모델로 예측

for index in np.random.choice(len(y_test), 3, replace = False):
    predicted = model.predict(X_test[index:index + 1])[0]
    label = y_test[index]
    result_label = np.where(label == np.amax(label))
    result_predicted = np.where(predicted == np.amax(predicted))

    title = "Label value = %s || Predicted value = %s " % (class_name[result_label[0][0]], class_name[result_predicted[0][0]])
    fig = plt.figure(1, figsize = (3,3))
    ax1 = fig.add_axes((0,0,.8,.8))
    ax1.set_title(title)
    images = X_test
    plt.imshow(images[index], cmap = plt.cm.gray_r, interpolation = 'nearest')
    plt.tight_layout()
    plt.show()